In [1]:
!nvidia-smi

Wed Dec  4 09:25:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install pypdf

In [3]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [4]:
!pip install sentence_transformers

In [5]:
!pip install llama-index==0.9.39

In [6]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt

In [7]:
!mkdir Data

In [7]:
documents = SimpleDirectoryReader("/content/Data/").load_data()

In [9]:
print(documents)

[Document(id_='600f8304-700e-4126-8376-2ea6d2cfd011', embedding=None, metadata={'page_label': '1', 'file_name': 'a4b95bc0bdb8361459a8b41bfc0ff317.pdf', 'file_path': '/content/Data/a4b95bc0bdb8361459a8b41bfc0ff317.pdf', 'file_type': 'application/pdf', 'file_size': 1311190, 'creation_date': '2024-12-04', 'last_modified_date': '2024-12-04', 'last_accessed_date': '2024-12-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='An ISO 9001, ISO 14001 & ISO \n45001 \nCertified Company \n  \n \n \n \n \n \n \n \n \nSARDA ENERGY & MINERALS LTD.\n18th November 2024  \n \nBSE Ltd \nThe Department of Corporate Services  \nPhiroze Jeejeebhoy Towers \nDalal Street – Mumbai 400 001 \n \nSecurity Code No.: 504614 \nNational Stock Exchange of India Ltd.  \nExchange Plaza,

In [10]:

system_prompt = """You are a pdf summarizer assistant. Your goal is to summarize pdf which may also include tabular columns, as
accurately as possible based on the instructions and context provided."""


# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [11]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write

### Model: https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1

In [12]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
pip install -U langchain-community

In [15]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)


In [16]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [17]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [18]:
query_engine = index.as_query_engine()
response = query_engine.query("Summarize the whole pdf in bullet points. keep it to the point and don't leave even a single information.")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [19]:
print(response)



* The document is a press release from Sarda Energy & Minerals Ltd. regarding their Q2 & H1 FY25 results.
* The company's production and sales for various products are provided in tables, including Iron Ore Pellet, Sponge Iron, Steel Billet, Wire Rod, H.B Wire, Ferro Alloys, Power (Thermal), Power (Hydro), and Coal.
* The tables include UOM, Quarter Half Year Ended, Q2FY25 Q1FY24 Q2FY24 Change %, H1FY25 H1FY24 YoY %, A B C QoQ YoY.
* The document also includes information about the newly acquired 2 x 300 MW SKS power plant and its impact on thermal power generation and sales.
* The document mentions that hydro power generation is seasonal and not comparable QoQ, and coal production is not comparable QoQ due to seasonal impact.
* The document is enclosed as an attachment.
